In [1]:
import pyproj
import pandas as pd

#import data

will need to change the filepath to your filepath

In [17]:
pantry_location = pd.read_csv('/mnt/datavault/alice/data4good_dvd/pantries.csv')

In [18]:
pantry_location.head()

,foodpantry,address,address2,city,state,zip,phone,county,lon,lat
0,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.888300,35.985724
1,Bethlehem Temple Apostolic Faith Church of Dur...,600 North Roxboro Street,NaN,Durham,NC,27704,(919) 683-1291,Durham,-78.894506,35.998290
2,Bethlehem United Holy Church Food Pantry,1311 Milton Road,NaN,Durham,NC,27712,(919) 815-2999,Durham,-78.918512,36.104674
3,Birchwood Learning Center Food Pantry,416 Walton Street,NaN,Durham,NC,27703,(919) 598-9885,Durham,-78.846948,35.981163
4,"CAARE, Inc.",214 Broadway Street,NaN,Durham,NC,27701,(919) 683-5300,Durham,-78.898764,36.001235


In [19]:
bus_stops = pd.read_csv('/mnt/datavault/alice/data4good_dvd/rdsstops.csv')

In [20]:
bus_stops.shape

(3467, 12)

In [21]:
bus_stops.head()

,code,description,url,parent_station_id,agency_ids,station_id,location_type,stop_id,routes,name,location.lat,location.lng
1,1240,NaN,NaN,NaN,12,NaN,stop,4025434,4000112,NC 55 at Colony Woods Dr,35.75749,-78.87623
2,1241,NaN,NaN,NaN,12,NaN,stop,4025438,4000112,NC 55 at Old Jenks Rd,35.76126,-78.87546
3,1242,NaN,NaN,NaN,12,NaN,stop,4025442,4000112,NC 55 at Lewey Dr,35.77369,-78.87318
4,1262,NaN,NaN,NaN,12,NaN,stop,4025474,"4000112, 4000117",Pine Plaza Dr at Summit Lake Dr,35.74599,-78.82045
5,1263,NaN,NaN,NaN,12,NaN,stop,4025478,"4000112, 4000117",Pine Plaza Dr at Shepherds Vineyard,35.74686,-78.82340


In [22]:
pantry_location.county.unique()

array(['Durham', 'Johnston', 'Orange', 'Wake'], dtype=object)

In [23]:
pantry_location.shape

(169, 10)

##exclude Johnston because Johnston does not have it's own busline

In [24]:
pantry_location = pantry_location.loc[~(pantry_location.county=='Johnston'), :]

In [25]:
pantry_location.shape

(136, 10)

##Cartesian product of pantries and bus stops to get distances

In [26]:
pantry_location['key'] = pd.Series([1]*pantry_location.shape[0])
bus_stops['key'] = pd.Series([1]*bus_stops.shape[0])
bus_by_pantry = pd.merge(pantry_location, bus_stops, on='key')

In [27]:
bus_by_pantry.shape

(357031, 23)

In [28]:
bus_by_pantry.head()

,foodpantry,address,address2,city,state,zip,phone,county,lon,lat,...,url,parent_station_id,agency_ids,station_id,location_type,stop_id,routes,name,location.lat,location.lng
0,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,...,NaN,NaN,12,NaN,stop,4025434,4000112,NC 55 at Colony Woods Dr,35.75749,-78.87623
1,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,...,NaN,NaN,12,NaN,stop,4025438,4000112,NC 55 at Old Jenks Rd,35.76126,-78.87546
2,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,...,NaN,NaN,12,NaN,stop,4025442,4000112,NC 55 at Lewey Dr,35.77369,-78.87318
3,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,...,NaN,NaN,12,NaN,stop,4025474,"4000112, 4000117",Pine Plaza Dr at Summit Lake Dr,35.74599,-78.82045
4,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,...,NaN,NaN,12,NaN,stop,4025478,"4000112, 4000117",Pine Plaza Dr at Shepherds Vineyard,35.74686,-78.82340


##Distance function

In [14]:
geod = pyproj.Geod(ellps='WGS84')
def distance(row):
            return geod.inv(row.bus_lng, row.bus_lat,
                            row.pantry_lon, row.pantry_lat)[2]

#get distances all except Johnston

In [29]:
bus_by_pantry.columns

Index([u'foodpantry', u'address', u'address2', u'city', u'state', u'zip',
       u'phone', u'county', u'lon', u'lat', u'key', u'code', u'description',
       u'url', u'parent_station_id', u'agency_ids', u'station_id',
       u'location_type', u'stop_id', u'routes', u'name', u'location.lat',
       u'location.lng'],
      dtype='object')

In [30]:
bus_by_pantry.columns = [u'foodpantry', u'address', u'address2', u'city', u'state', u'zip',
       u'phone', u'county', u'pantry_lon', u'pantry_lat', u'key', u'code', u'description',
       u'url', u'parent_station_id', u'agency_ids', u'station_id',
       u'location_type', u'stop_id', u'routes', u'name', u'bus_lat',
       u'bus_lng']

In [31]:
bus_by_pantry_small = bus_by_pantry.loc[:, [u'bus_lng', u'bus_lat', u'pantry_lon',
       u'pantry_lat']] #just want distances

In [33]:
bus_by_pantry_small.shape

(357031, 4)

In [34]:
#drop missing lat/long values

bus_by_pantry.dropna(subset=[u'bus_lat', u'bus_lng', u'pantry_lon',
       u'pantry_lat'], inplace=True)
bus_by_pantry_small.dropna(subset=[u'bus_lng', u'bus_lat', u'pantry_lon',
       u'pantry_lat'], inplace=True)

In [35]:
bus_by_pantry.shape

(357031, 23)

In [36]:
bus_by_pantry_small.shape

(357031, 4)

In [37]:
bus_by_pantry_small.head()

,bus_lng,bus_lat,pantry_lon,pantry_lat
0,-78.87623,35.75749,-78.8883,35.985724
1,-78.87546,35.76126,-78.8883,35.985724
2,-78.87318,35.77369,-78.8883,35.985724
3,-78.82045,35.74599,-78.8883,35.985724
4,-78.82340,35.74686,-78.8883,35.985724


In [38]:
bus_by_pantry['distance_meter'] = bus_by_pantry_small.apply(distance, 1) #get distance in meters
bus_by_pantry['distance_miles'] =bus_by_pantry.distance_meter/1000/1.60934 #convert to miles

In [39]:
pd.set_option('display.max_columns', 500)
bus_by_pantry.head()

,foodpantry,address,address2,city,state,zip,phone,county,pantry_lon,pantry_lat,key,code,description,url,parent_station_id,agency_ids,station_id,location_type,stop_id,routes,name,bus_lat,bus_lng,distance_meter,distance_miles
0,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,1,1240,NaN,NaN,NaN,12,NaN,stop,4025434,4000112,NC 55 at Colony Woods Dr,35.75749,-78.87623,25347.554909,15.750280
1,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,1,1241,NaN,NaN,NaN,12,NaN,stop,4025438,4000112,NC 55 at Old Jenks Rd,35.76126,-78.87546,24932.785692,15.492553
2,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,1,1242,NaN,NaN,NaN,12,NaN,stop,4025442,4000112,NC 55 at Lewey Dr,35.77369,-78.87318,23566.225224,14.643410
3,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,1,1262,NaN,NaN,NaN,12,NaN,stop,4025474,"4000112, 4000117",Pine Plaza Dr at Summit Lake Dr,35.74599,-78.82045,27296.809521,16.961493
4,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,1,1263,NaN,NaN,NaN,12,NaN,stop,4025478,"4000112, 4000117",Pine Plaza Dr at Shepherds Vineyard,35.74686,-78.82340,27143.967386,16.866521


In [40]:
bus_less_than_1mi = bus_by_pantry.loc[bus_by_pantry.distance_miles<1, :]

then I will have distances less than 1 mile and all I will need to do is check which pantries are in this list and which are not-- aggregate by county and compare to aggregate of all by county

In [41]:
bus_less_than_1mi.head()

,foodpantry,address,address2,city,state,zip,phone,county,pantry_lon,pantry_lat,key,code,description,url,parent_station_id,agency_ids,station_id,location_type,stop_id,routes,name,bus_lat,bus_lng,distance_meter,distance_miles
158,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,1,5006,NaN,NaN,NaN,24,NaN,stop,4049730,4000079,Alston Ave at Durham Freeway,35.98103,-78.89091,571.594021,0.355173
159,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,1,5008,NaN,NaN,NaN,"12, 24",NaN,stop,4049734,"4000079, 4000116",S Alston Ave at Massey Ave,35.97815,-78.89219,910.722078,0.565898
167,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,1,5024,NaN,NaN,NaN,24,NaN,stop,4049778,"4000085, 4003062",2309 Angier Ave,35.98067,-78.87443,1370.788797,0.851771
176,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,1,5038,NaN,NaN,NaN,24,NaN,stop,4049818,"4000083, 4000086, 4003058, 4005298",Liberty St at Queen St,35.99365,-78.89555,1095.824666,0.680916
177,Asbury Temple United Methodist Church,201 Alston Avenue,NaN,Durham,NC,27701,(919) 688-4578,Durham,-78.8883,35.985724,1,5039,NaN,NaN,NaN,24,NaN,stop,4049822,"4000088, 4004490",Fayetteville St at E Piedmont Ave,35.98433,-78.89912,987.962075,0.613893


In [42]:
bus_less_than_1mi.shape

(4928, 25)

In [95]:
bus_less_than_1mi.to_csv('/mnt/datavault/alice/data4good/distances_pantries_to_busstop.csv', index=False)

In [43]:
pantries_with_bus = bus_less_than_1mi.drop_duplicates('foodpantry')

In [44]:
pantries_with_bus.shape

(86, 25)

#So only 86 pantries have buses that serve them within a mile out of 136 pantries, not counting Johnston County (with Johnston there are 169 pantries, total)-- that's about 2/3 of pantries being served

In [45]:
pantry_location.shape

(136, 11)

In [83]:
169-136

33

In [46]:
86.0/136.0

0.6323529411764706

##Now want to look at pantries by county-- the proportion that have bus stops

In [48]:
pantry_location.columns

Index([u'foodpantry', u'address', u'address2', u'city', u'state', u'zip',
       u'phone', u'county', u'lon', u'lat', u'key'],
      dtype='object')

In [52]:
pantry_location.groupby('county').count()['foodpantry']

county
Durham    39
Orange     6
Wake      91
Name: foodpantry, dtype: int64

In [53]:
pantries_with_bus.groupby('county').count()['foodpantry']

county
Durham    37
Orange     4
Wake      45
Name: foodpantry, dtype: int64

In [62]:
pantries_county = pd.DataFrame(pantry_location.groupby('county').count()['foodpantry'])
pantries_bus_county = pd.DataFrame(pantries_with_bus.groupby('county').count()['foodpantry'])

In [63]:
pantries_county

,foodpantry
county,
Durham,39
Orange,6
Wake,91


In [66]:
county_bus_availability = pd.merge(pantries_county, pantries_bus_county, left_index=True, right_index=True, suffixes=('_all','_with_bus'))

In [67]:
county_bus_availability

,foodpantry_all,foodpantry_with_bus
county,,
Durham,39,37
Orange,6,4
Wake,91,45


In [70]:
import numpy as np

In [79]:
county_bus_availability['percent_bus']=100*county_bus_availability.foodpantry_with_bus/county_bus_availability.foodpantry_all.apply(float)

##Percent availability by county

In [80]:
county_bus_availability

,foodpantry_all,foodpantry_with_bus,percent_bus
county,,,
Durham,39,37,94.871795
Orange,6,4,66.666667
Wake,91,45,49.450549


In [81]:
county_bus_availability.reset_index(inplace=True)

In [82]:
county_bus_availability

,county,foodpantry_all,foodpantry_with_bus,percent_bus
0,Durham,39,37,94.871795
1,Orange,6,4,66.666667
2,Wake,91,45,49.450549


In [88]:
johnston = pd.Series({'county': 'Johnston', 'foodpantry_all':33, 'foodpantry_with_bus': 0, 'percent_bus':0})

In [93]:
county_bus_availability=county_bus_availability.append(pd.DataFrame(johnston).T, ignore_index=True)

In [94]:
county_bus_availability.to_csv('/mnt/datavault/alice/data4good/pantry_bus_availability_by_county.csv', index=False)